
## 🔧 คำแนะนำ: หากคุณเปลี่ยน CSV ให้ตรวจสอบและแก้ไขดังนี้

หากคุณเปลี่ยนไฟล์ CSV ที่ใช้ในการเทรนโมเดล (เช่น ข้อมูลมีคอลัมน์ต่างออกไป หรือชื่อไม่เหมือนเดิม)  
ให้แก้ไขในตำแหน่งต่อไปนี้ใน Notebook:

1. **Cell ที่โหลดข้อมูล (read_csv)**  
   - แก้ชื่อไฟล์ CSV ที่จะใช้ (`pd.read_csv("ชื่อไฟล์ใหม่.csv")`)
   - ตรวจสอบว่าคอลัมน์ที่ใช้ เช่น `Close`, `Open`, `Date` หรือชื่ออื่น มีอยู่จริงหรือไม่  
     หากไม่เหมือนกัน ให้แก้ชื่อคอลัมน์ในบรรทัดที่มี `data['Close']` หรือ `data[['Open', 'Close']]`

2. **Cell ที่เตรียมข้อมูล (Data Preprocessing)**  
   - หากจำนวนฟีเจอร์หรือคอลัมน์เปลี่ยนไป ต้องแก้โค้ดในส่วนที่กำหนด `feature_columns`  
   - ถ้าไม่มีคอลัมน์ `Date` ต้องลบหรือแก้ส่วนที่ใช้ `pd.to_datetime()`

3. **Cell ที่สร้างโมเดล (Model Building)**  
   - ไม่ต้องแก้ หากเพียงเปลี่ยนข้อมูล แต่ถ้า shape ของข้อมูลเปลี่ยน (จำนวน feature เพิ่ม/ลด)  
     ให้แก้ค่าของ `input_shape` ใน layer แรก เช่น `input_shape=(time_steps, num_features)`

4. **Cell ที่เทรนโมเดล (Model Training)**  
   - ตรวจสอบว่าชุดข้อมูล `X_train`, `y_train` มีขนาดสอดคล้องกัน (ใช้ `.shape` เช็คได้)
   - หากเทรนไม่ได้ ให้ตรวจสอบการสเกลข้อมูล (`MinMaxScaler`) ว่าครอบคลุมทุกคอลัมน์หรือไม่

5. **Cell ที่พล็อตกราฟผลลัพธ์ (Visualization)**  
   - ถ้าชื่อคอลัมน์เปลี่ยน ต้องแก้ส่วน `plt.plot(data['Close'])` ให้ตรงกับคอลัมน์ใหม่

> 💡 สรุป: หากเปลี่ยน CSV ต้องตรวจสอบคอลัมน์, จำนวน feature, และการสเกลข้อมูลให้ตรงกับโครงสร้างใหม่


In [ ]:
import os, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import joblib

# reproducibility
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

for dirname, _, filenames in os.walk('datasets/northSulawesi/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# 🔸 ถ้าข้อมูลมีคอลัมน์เพิ่ม/ลด ต้องปรับช่วงสเกลให้ครอบคลุมคอลัมน์ใหม่ทั้งหมด

# 🔸 ถ้าจำนวน feature เปลี่ยน ให้แก้ input_shape=(time_steps, num_features) ให้ตรง


In [ ]:
FILE = 'datasets/northSulawesi/cabai_complete.csv'  # เปลี่ยนชื่อไฟล์ที่นี่
df = pd.read_csv(FILE, parse_dates=['date'] if 'date' in pd.read_csv(FILE, nrows=0).columns else None)

# ถ้ามีคอลัมน์วันที่ ให้เรียงลำดับตามวันที่
if 'date' in df.columns:
    df = df.sort_values('date').reset_index(drop=True)

print("Shape:", df.shape)
display(df.head())
print("\nColumn & Type:")
print(df.dtypes)
# 🔸 ถ้าเปลี่ยนชื่อไฟล์ CSV ให้แก้ที่บรรทัดนี้ เช่น pd.read_csv('new_data.csv')
# 🔸 ตรวจสอบว่าชื่อคอลัมน์ (เช่น 'Close', 'Open') ตรงกับในไฟล์ใหม่


การวิเคราะห์ข้อมูลเชิง Exploratory Data Analysis (EDA)
- ดูสถิติสรุป
- นับข้อมูลที่หายไปต่อคอลัมน์

In [ ]:
display(df.describe(include='all').T)
print("\nMissing values per column:")
print(df.isna().sum())

# Visual: price trend (if there is a 'prices' column)
# ภาพ: แนวโน้มราคา (ถ้ามีคอลัมน์ 'ราคา')
if 'prices' in df.columns:
    plt.figure(figsize=(12,4))
    plt.plot(df['prices'])
    plt.title("Price Trends (prices)")
    plt.ylabel("Price")
    plt.show()
# 🔸 ถ้าชื่อคอลัมน์ใน CSV เปลี่ยน ต้องแก้ชื่อใน plt.plot(...) ให้ตรงกับคอลัมน์ใหม่


วิศวกรรมคุณลักษณะ (ความล่าช้า, การหมุนเวียน) โดยไม่ลดค่า NaN
หลักการ:
- ค่าเฉลี่ยแบบหมุนเวียนคำนวณโดยใช้ min_periods=1 เพื่อหลีกเลี่ยงค่า NaN เริ่มต้น
- ความล่าช้าเกิดขึ้นจากการเลื่อน; เติมค่า NaN เริ่มต้นอย่างระมัดระวัง (เติมด้วยราคาปัจจุบัน)
- คุณลักษณะภายนอก (cpi, usd_idr, สภาพอากาศ) คำนวณโดยใช้ ffill->bfill
- เพิ่มคุณลักษณะที่ได้มาซึ่งมีประโยชน์ (การเปลี่ยนแปลงเปอร์เซ็นต์)

In [ ]:
# make sure prices are there
# ตรวจสอบให้แน่ใจว่ามีราคา
if 'prices' not in df.columns:
    raise ValueError("Data does not contain a 'prices' column — adjust the column names.")

# Copy working df (so as not to modify the original)
d = df.copy()

# Fill in missing data for exogenous data with forward fill and then back fill (assuming the missing data is small and time series in nature)
exo_cols = [c for c in ['cpi', 'usd_idr', 'Temperature', 'Curah Hujan', 'Kelembapan'] if c in d.columns]
if exo_cols:
    d[exo_cols] = d[exo_cols].ffill().bfill()

# Rolling means — use min_periods=1 so that early rows have a value
d['rolling_mean_7'] = d['prices'].rolling(window=7, min_periods=1).mean()
d['rolling_mean_30'] = d['prices'].rolling(window=30, min_periods=1).mean()

# Lag features
d['lag_1'] = d['prices'].shift(1)
d['lag_7'] = d['prices'].shift(7)
d['lag_30'] = d['prices'].shift(30)

# NaN filling strategy for lag: fill initial NaNs with current price values (conservative)
# Rationale: on the first day there is no history, it is safer to assume the historical value = the price of that day
d['lag_1'] = d['lag_1'].fillna(d['prices'])
d['lag_7'] = d['lag_7'].fillna(d['prices'])
d['lag_30'] = d['lag_30'].fillna(d['prices'])

# Delta / pct change
d['pct_change_1'] = d['prices'].pct_change().fillna(0)
d['pct_change_7'] = d['prices'].pct_change(periods=7).fillna(0)

# day features
if 'date' in d.columns:
    d['weekday'] = d['date'].dt.dayofweek  # 0=Mon..6=Sun
    d['day_in_month'] = d['date'].dt.day
    d['month'] = d['date'].dt.month
else: 
    # if there is no date, try using the day_in_month column if available
    if 'day_in_month' not in d.columns:
        d['day_in_month'] = 0

# Check results
display(d.head(10))
print("\nMissing after imputation (should be 0 for features used):")
print(d[['rolling_mean_7','rolling_mean_30','lag_1','lag_7','lag_30','pct_change_1']].isna().sum())

In [ ]:
features = [
    'prices', 'rolling_mean_7', 'rolling_mean_30', 'lag_1', 'lag_7', 'lag_30',
    'pct_change_1', 'pct_change_7', 'weekday', 'day_in_month'
]
# เพิ่มปัจจัยภายนอกถ้ามี
for ex in exo_cols:
    features.append(ex)

# ให้แน่ใจว่ามีทุกอย่างอยู่
missing = [c for c in features if c not in d.columns]
if missing:
    raise ValueError("Kolom hilang: " + ", ".join(missing))

df_feat = d[features].copy()
display(df_feat.head())

# ความสัมพันธ์ของคุณลักษณะ (ตัวเลขเท่านั้น)
plt.figure(figsize=(10,8))
sns.heatmap(df_feat.corr(), annot=True, fmt=".2f", cmap='coolwarm')
plt.title("Correlation (final features)")
plt.show()

การปรับขนาดและการสร้างลำดับ (ป้องกันการรั่วไหลของข้อมูล) (data leakage)
- ใช้ window_size = 30
- ปรับตัวปรับขนาดให้พอดีกับแถวที่จะปรากฏในลำดับการฝึกเท่านั้น
- แปลงข้อมูลทั้งหมด แล้วสร้างลำดับ X, y

In [ ]:
raw_values = df_feat.values  # shape (n_rows, n_features)
n_rows, n_features = raw_values.shape
window_size = 30

# Count sequences available
seq_count = n_rows - window_size
if seq_count <= 0:
    raise ValueError("Data tidak cukup untuk window_size yang dipilih")

# Training sequences count (time-based split)
train_seq_count = int(seq_count * 0.8)
rows_for_scaler = train_seq_count + window_size  # rows needed to construct training windows
print(f"n_rows={n_rows}, seq_count={seq_count}, train_seq_count={train_seq_count}, rows_for_scaler={rows_for_scaler}")

scaler = MinMaxScaler()
scaler.fit(raw_values[:rows_for_scaler, :])  # fit only on training rows (prevent leakage)

scaled_all = scaler.transform(raw_values)     # transform all rows for convenience

# Create sequences
def create_sequences_from_scaled(scaled_array, window):
    X, y = [], []
    for i in range(len(scaled_array) - window):
        X.append(scaled_array[i:i+window])
        y.append(scaled_array[i+window, 0])  # target: kolom 'prices' (index 0)
    return np.array(X), np.array(y)

X, y = create_sequences_from_scaled(scaled_all, window_size)
print("Total sequences:", X.shape, y.shape)

# Split
X_train, X_test = X[:train_seq_count], X[train_seq_count:]
y_train, y_test = y[:train_seq_count], y[train_seq_count:]
print("Shapes -> X_train:", X_train.shape, "X_test:", X_test.shape)
# 🔸 ถ้าข้อมูลมีคอลัมน์เพิ่ม/ลด ต้องปรับช่วงสเกลให้ครอบคลุมคอลัมน์ใหม่ทั้งหมด


Model architecture: LSTM_v2_Imputed
Architecture:
- LSTM(128, return_sequences=True)
- Dropout(0.3)
- LSTM(64)
- Dropout(0.3)
- Dense(64, relu)
- Dense(1)

Rationale:
- Two LSTM layers help the model capture complex temporal patterns
- Dropout for regularization
- Dense layers help with non-linear mapping before output

In [ ]:
model = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    LSTM(64, return_sequences=False),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
# 🔸 ถ้าจำนวน feature เปลี่ยน ให้แก้ input_shape=(time_steps, num_features) ให้ตรง


Training
- EarlyStopping: stop if val_loss doesn't improve
- ReduceLROnPlateau: reduce LR if it plateaus
- ModelCheckpoint: save the best model
- Goal: optimize the model while avoiding overfitting

In [ ]:
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1)
ckpt_path = "models/LSTM_v2_Imputed_best.h5"
mc = ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True, verbose=1)

history = model.fit(
    X_train, y_train,
    epochs=150,
    batch_size=32,
    validation_split=0.2,
    callbacks=[es, rlr, mc],
    verbose=1
)

In [ ]:
"""
Load best model & evaluation
- Muat model terbaik dari checkpoint
- Prediksi X_test (scaled)
- Inverse transform predictions & actual ke skala asli
- Hitung MSE, MAE, RMSE
"""
if os.path.exists(ckpt_path):
    model = load_model(ckpt_path)

# Predict (scaled)
pred_scaled = model.predict(X_test)

# inverse transform helper
def inv_transform_target(scaled_target_array, scaler, n_features):
    """
    Inverse transform helper to return prices to their original scale.
    scaled_target_array: an (N,) or (N,1) array containing the scaled targets (column 0)
    scaler: the fitted scaler
    n_features: the total number of features
    """
    scaled_target_array = scaled_target_array.flatten()  # pastikan bentuk (N,)
    dummy = np.zeros((len(scaled_target_array), n_features))
    dummy[:, 0] = scaled_target_array
    return scaler.inverse_transform(dummy)[:, 0]

y_test_inv = inv_transform_target(y_test, scaler, n_features)
pred_inv = inv_transform_target(pred_scaled, scaler, n_features)

mse = mean_squared_error(y_test_inv, pred_inv)
mae = mean_absolute_error(y_test_inv, pred_inv)
rmse = np.sqrt(mse)

print(f"MSE: {mse:.2f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}")

In [ ]:
"""
Visualisasi:
- Actual vs Predicted
- Error distribution
- Training/validation loss
"""
plt.figure(figsize=(12,5))
plt.plot(y_test_inv, label='Actual', color='blue')
plt.plot(pred_inv, label='Predicted', color='orange')
plt.title("Actual vs Predicted (Test Set)")
plt.legend()
plt.show()

# Error distribution
errors = y_test_inv - pred_inv
plt.figure(figsize=(8,4))
sns.histplot(errors, bins=40, kde=True)
plt.title("Error distribution (Actual - Predicted)")
plt.show()

# Loss curves
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.yscale('log')
plt.legend()
plt.title("Training & Validation Loss")
plt.show()
# 🔸 ถ้าชื่อคอลัมน์ใน CSV เปลี่ยน ต้องแก้ชื่อใน plt.plot(...) ให้ตรงกับคอลัมน์ใหม่


"""
Save model & scaler
- Save LSTM_v2_Imputed model and scaler
- Log simple metadata (version, date)

MODEL_OUT = "/kaggle/working/LSTM_v2_Imputed.h5"
SCALER_OUT = "/kaggle/working/price_scaler_v2.save"
model.save(MODEL_OUT)
joblib.dump(scaler, SCALER_OUT)

# Write metadata
meta = {
    "model_name": "LSTM_v2_Imputed",
    "window_size": window_size,
    "features": features,
    "n_features": n_features
}
import json
with open("/kaggle/working/LSTM_v2_Imputed_metadata.json", "w") as f:
    json.dump(meta, f, indent=2)
print("Saved model, scaler, and metadata.")
"""


"""
Function predict_next_day
- Input: model, scaler, raw_values ​​(original features in original scale), window_size
- Output: predicted price (original scale)
- Note: make sure the final raw_values ​​have the features we have defined

def predict_next_day(model, scaler, raw_values_array, window_size=30):
    # raw_values_array: ndarray (n_rows, n_features) in original scale (same order as 'features')
    last_window_raw = raw_values_array[-window_size:, :]
    last_window_scaled = scaler.transform(last_window_raw)
    X_input = last_window_scaled.reshape(1, window_size, last_window_scaled.shape[1])
    pred_scaled = model.predict(X_input)
    # inverse transform prediction
    dummy = np.zeros((1, n_features))
    dummy[0,0] = pred_scaled.flatten()[0]
    pred_inv = scaler.inverse_transform(dummy)[0,0]
    return float(pred_inv)

# usage example (raw_values ​​is df_feat.values ​​from Cell 5)
pred_next = predict_next_day(model, scaler, df_feat.values, window_size=window_size)
print("Tomorrow's price prediction (1-step):", pred_next)

"""

"""
Simulasi update harian:
- Simulasikan admin memasukkan harga hari ini
- Bangun baris baru dengan fitur (disederhanakan: gunakan last row untuk exogenous)
- Append ke raw_values (tidak overwrite file asli) lalu panggil predict_next_day

# contoh: input manual
harga_hari_ini = float(input("Masukkan harga hari ini (simulasi): "))

# gunakan row terakhir sebagai basis fitur
last_row_raw = raw_values[-1].copy()  # raw_values diambil dari Cell 6 awal (df_feat.values)
new_row = last_row_raw.copy()
new_row[0] = harga_hari_ini

# update rolling & lag di new_row: (opsional, sederhana)
# lebih baik compute new_row fitur sebenarnya via helper fungsi yang memakai history + sumber exogenous
raw_values_upd = np.vstack([raw_values, new_row])

pred_after_input = predict_next_day(model, scaler, raw_values_upd, window_size)
print("Prediksi harga besok setelah input harga terbaru:", pred_after_input)
"""